针对上述任意样本，比如SGI75460067的count/tpm/cpm，会得到如下的结果
|基因名称|	Hisat-rf结果|	Hisat-fr结果|	Star结果|
|-|-|-|-|
|ABC	|150	|140	|600|
|DEF	|120000	|10000	|2600|
|GHI	|0	|1|200|

算了，算了，还是手动合并算了，不行啊 还是要写代码

In [ ]:
# import os
# import pandas as pd
# #"D:\\githubku\\my_jupyter\\20230206_tpm_count\\fina/12"文件夹下所有文件夹名，保存为list
# path = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/12"
# file_list = os.listdir(path)
# # 读取D:\\githubku\\my_jupyter\\20230206_tpm_count\\final中的count_all.xlsx，cpm_all.xlsx，tpm_all.xlsx
# count_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/count_all.xlsx")
# cpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/cpm_all.xlsx")
# tpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/tpm_all.xlsx")

In [ ]:
import pandas as pd
import os
ref = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\/merge1v1.txt",sep="\t")
##每种类型所包含的基因名称，以及

In [ ]:
id="SGI75460067"

In [ ]:
# D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\tpm\\tpm

In [ ]:
inputpath1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\sample_RF_star\\cpm\\cpm"+id+".txt"
outputpath1="D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\cpm\\"+id+"\\"

In [ ]:
os.makedirs(os.path.dirname(outputpath1), exist_ok=True)

In [ ]:
#保留Gene Name	Gene type列
ref = ref[["Gene Name","Gene type"]]
#去除gene name重复行
ref = ref.drop_duplicates(subset="Gene Name",keep="first")

In [ ]:
#读入D:\githubku\my_jupyter\20230206_tpm_count\final\12\SGI75460067\0067count.txt文件
count = pd.read_csv(inputpath1,sep="\t")

In [ ]:
#ref和count合并，以gene name为键，保留ref中的gene type
count1 = pd.merge(ref,count,left_on="Gene Name",right_on="Gene Name_RFfr",how="right")


In [ ]:
count1

In [ ]:
#将count1中的gene type列，转换为行索引
count1 = count1.set_index("Gene type")


In [ ]:
#输出count1中gene type为空的行
count1.loc[count1.index.isnull()]

In [ ]:
#获取count1中索引，保存为list，去重
index_name = list(set(count1.index))
#遍历index_name,获取count1中对应索引的数据，保存到文件中
for i in index_name:
    count1[count1.index==i].to_csv(outputpath1+"\\"+id+".%s.txt"%i,sep="\t")


## 计算pearson相关系数和p     使用r语言

In [ ]:
#创建一个空的数据框
count_all = pd.DataFrame()
#遍历file_list，读取文件，保存到count_all中,文件名作为索引
file_list = os.listdir("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\tpm")
for i in file_list:
    file = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\tpm\\"+i,sep=",",index_col=0)
    #将文件名设置为多级索引
    #删除i包含的sum.txt
    i = i.replace("sum.txt","")
    file.index = pd.MultiIndex.from_product([[i],file.index])
    #将file添加到count_all中
    count_all = pd.concat([count_all,file],axis=0)
count_all
    

In [ ]:
# 创建一个空的数据框
count_all1 = pd.DataFrame()
count_temp = pd.DataFrame()
#在多级索引中分别查找"Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"的数据
for i in ["Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"]:
    count_temp=count_all.loc[count_all.index.get_level_values(1)==i]
    # 删除count_temp中第一个索引
    count_temp = count_temp.droplevel(1)
    # count_temp设置为多级索引
    count_temp.index = pd.MultiIndex.from_product([[i],count_temp.index])
    count_all1 = pd.concat([count_all1,count_temp],axis=0)
# count_all1输出到文件中
count_all1.to_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\final\\tpm_all.txt",sep="\t")


## 合并tpm count cpm

In [1]:
import pandas as pd
import os

In [ ]:
#合并cpm 和 tpm
# 创建空dataframe
cpm_1 = pd.DataFrame()
cpm_2 = pd.DataFrame()
path_filefolder1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\RFfr\\tpm"
path_filefolder2="D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\star\\tpm"
outputfilefolder = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\RF_star\\tpm"
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\cpm中的文件
file_list1 = os.listdir(path_filefolder1)
for i1 in file_list1:
    file1 = pd.read_csv(path_filefolder1+"\\"+i1,sep=",")
    #按照Gene Name升序排列
    file1 = file1.sort_values(by="Gene Name")
    #将文件名设置为多级索引
    #删除i包含的sum.txt
    i1 = i1.replace("Name_tPM.csv","")
    file1.index = pd.MultiIndex.from_product([[i1],file1.index])
    #将file添加到count_all中
    cpm_1 = pd.concat([cpm_1,file1],axis=0)
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\star\cpm中的文件
file_list2 = os.listdir(path_filefolder2)
for i2 in file_list2:
    file2 = pd.read_csv(path_filefolder2+"\\"+i2,sep=",")
    #将文件名设置为多级索引
    file2 = file2.sort_values(by="Gene Name")
    #删除i包含的sum.txt
    i2 = i2.replace("Name_tPM.csv","")
    file2.index = pd.MultiIndex.from_product([[i2],file2.index])
    #将file添加到count_all中
    cpm_2 = pd.concat([cpm_2,file2],axis=0)
    
        

In [ ]:
# cpm_1列名修改为Gene Name_RFfr,tpm_RFfr
cpm_1.columns = ["Gene Name_RFfr","tpm_RFfr"]
# cpm_2列名修改为Gene Name_star,tpm_star
cpm_2.columns = ["Gene Name_star","tpm_star"]

In [ ]:
# cpm_1列名修改为Gene Name_RFfr,count_RFfr,cpm_RFfr
cpm_1.columns = ["Gene Name_RFfr","count_RFfr","cpm_RFfr"]
# cpm_2列名修改为Gene Name_star,count_star,cpm_star
cpm_2.columns = ["Gene Name_star","count_star","cpm_star"]

In [ ]:
#将cpm_1和cpm_2按照level_values(1)合并
cpm_12 = cpm_1.join(cpm_2)

In [ ]:
cpm_12.loc[cpm_12.index.get_level_values(0)==i]

In [ ]:
# 输出cpm_12以及索引对应的数据
# 创建一个空的数据框
cpm_12_1 = pd.DataFrame()
id = cpm_12.index.get_level_values(0)
#遍历cpm_12的索引，获取对应的数据，保存到cpm_12_1中，依次输出
for i in set(cpm_12.index.get_level_values(0)):
    cpm_12_1 = cpm_12.loc[cpm_12.index.get_level_values(0)==i]
    cpm_12_1.to_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\sample_RF_star\\tpm\\tpm"+i+".txt",sep="\t")

In [48]:

import pandas as pd
import os
## 合并同一批样本的tpm  
inputfilefolder = "C:\\Users\\zhangyifan1\\Desktop\\work\\0217\\name_tpm\\"
outputfile= "C:\\Users\\zhangyifan1\\Desktop\\work\\0217\\jingshen0207_name_tpm.csv"
#遍历文件夹的文件，将文件名保存到file_list中
file_list = os.listdir(inputfilefolder)
# 创建空dataframe
tpm_1 = pd.DataFrame()
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\tpm中的文件,按照Gene Name合并
for i in file_list:
    file = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
    #按照Gene Name升序排列
    file = file.sort_values(by="Gene Name")
    #文件名替换TPM作为列名
    i = i.replace(".Name_tPM.csv","")
    file.columns = ["Gene Name",i]
    #依次将file添加到tpm_1中
    tpm_1 = pd.concat([tpm_1,file],axis=1)
# 创建一个新数据框，将tpm_1的第一列设为索引
tpm_1_1 = pd.DataFrame(tpm_1.iloc[:,0])
#将tpm_1中列名不为  Gene Name的列添加到tpm_1_1中
tpm_1_1 = pd.concat([tpm_1_1,tpm_1.loc[:,tpm_1.columns != "Gene Name"]],axis=1)
#将Gene Name设置为索引
tpm_1_1.index = tpm_1_1["Gene Name"]
#删除Gene Name列
tpm_1_1 = tpm_1_1.drop("Gene Name",axis=1)
#输出tpm_1_1
tpm_1_1.to_csv(outputfile,sep=",")

In [35]:
#输出不为列名不为Gene Name的列
tpm_1.loc[:,tpm_1.columns!="Gene Name"]

,UHR+r-J-010-BT513-S,UHR+r-J-050-BT513-S
0,0.081610,0.076459
1,0.049236,0.092194
2,0.166203,0.142967
3,9.972236,9.610504
4,0.044740,0.068698
...,...,...
45093,0.624894,0.749918
45094,0.000000,0.000000
45095,0.413722,0.332935
45096,0.699649,0.556337


In [28]:
#获取tpm_1第二列的内容
tpm_1.iloc[:,1]

0        0.081610
1        0.049236
2        0.166203
3        9.972236
4        0.044740
           ...   
45093    0.624894
45094    0.000000
45095    0.413722
45096    0.699649
45097    0.000000
Name: UHR+r-J-010-BT513-S, Length: 45098, dtype: float64

In [24]:
import pandas as pd
import os
## 合并同一批样本的tpm  
inputfilefolder = "C:\\Users\\zhangyifan1\\Desktop\\0216\\0210\\name_tpm\\"
outputfile= "C:\\Users\\zhangyifan1\\Desktop\\name_tpm\\0210name_tpm.csv"
#遍历文件夹的文件，将文件名保存到file_list中
file_list = os.listdir(inputfilefolder)
# 创建空dataframe
tpm_1 = pd.DataFrame()
cnt=1
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\tpm中的文件,按照Gene Name合并
#第一个文件保存为file，之后的文件保存为file2
for i in file_list:
    if cnt ==1:
        file = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
        #按照Gene Name升序排列
        file = file.sort_values(by="Gene Name")
        #文件名替换TPM作为列名
        i = i.replace("Name_tPM.csv","")
        file.columns = ["Gene Name",i]
        #依次将file横向添加到tpm_1中
        tpm_1 = pd.concat([tpm_1,file],axis=1)
        #将tpm_1第一列设为索引
        tpm_1 = tpm_1.set_index("Gene Name")
        cnt +=1
    else:
        file2 = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
        #按照Gene Name升序排列
        file2 = file.sort_values(by="Gene Name")
        #文件名替换TPM作为列名
        i = i.replace("Name_tPM.csv","")
        file2.columns = ["Gene Name",i]
        # 将file2合并到tpm_1中
        # 将第一列设置为索引
        file2 = file2.set_index("Gene Name")
        file = pd.concat([file,file2[i]],axis=1)
        cnt+=1
tpm_1.to_csv(outputfile,sep=",")
    